<a href="https://www.kaggle.com/code/xiomaraariasa/llm-predictor?scriptVersionId=250106612" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/featuredatasets/full_feature_dataset.csv


In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"
!pip install -q google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 2.9 MB/s eta 0:00:00


In [3]:

from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [4]:

from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)


In [5]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

flash = genai.GenerativeModel('gemini-1.5-flash')

In [6]:
df = pd.read_csv("/kaggle/input/featuredatasets/full_feature_dataset.csv")

# 🧠 Function to generate prompt and analyze the match
def analyze_match(row):
    prompt = f"""
You are a data analyst reviewing NCAA Tournament match predictions.

Match Information:
- Season: {row['Season']}, Gender: {row['Gender']}
- Winning Team ID: {row['WTeamID']}, Losing Team ID: {row['LTeamID']}
- Seeds → Winner: {row['SeedA']}, Loser: {row['SeedB']} (Difference: {row['SeedDiff']})
- Winner Stats → Win/Loss: {row['W_WinLoss']}, PPG: {row['W_PPG']}, Allowed Points: {row['W_PtsAllowed']}, Avg Margin: {row['W_Margin']}
- Loser Stats → Win/Loss: {row['L_WinLoss']}, PPG: {row['L_PPG']}, Allowed Points: {row['L_PtsAllowed']}, Avg Margin: {row['L_Margin']}
- Feature Differences → WinLossDiff: {row['Diff_WinLoss']}, MarginDiff: {row['Diff_Margin']}, PPGDiff: {row['Diff_PPG']}, PtsAllowedDiff: {row['Diff_PtsAllowed']}

Your tasks:
1. Does it make sense that team {row['WTeamID']} won, based on the data?
2. Which statistical differences were likely most influential?
3. Suggest one new feature that could improve future model predictions.
"""
    try:
        response = flash.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"❌ Error: {e}"

# 🔁 Run the analysis on the first 5 rows (adjust if needed)
df['gemini_analysis'] = df.head(5).apply(analyze_match, axis=1)


df.head()

,Season,Gender,WTeamID,LTeamID,SeedA,SeedB,SeedDiff,W_WinLoss,W_PPG,W_PtsAllowed,W_Margin,L_WinLoss,L_PPG,L_PtsAllowed,L_Margin,Diff_WinLoss,Diff_Margin,Diff_PPG,Diff_PtsAllowed,gemini_analysis
0,2015,M,1214,1264,16,16,0,0.484848,67.151515,68.000000,-0.848485,0.593750,69.875000,67.500000,2.375000,-0.108902,-3.223485,-2.723485,0.500000,"1. **Does it make sense that team 1214 won, ba..."
1,2015,M,1279,1140,11,11,0,0.625000,72.625000,67.500000,5.125000,0.718750,81.843750,72.937500,8.906250,-0.093750,-3.781250,-9.218750,-5.437500,"Based on the provided data, it seems counterin..."
2,2015,M,1173,1129,11,11,0,0.757576,68.212121,60.909091,7.303030,0.741935,69.870968,60.935484,8.935484,0.015640,-1.632454,-1.658847,-0.026393,## Analysis of 2015 NCAA Men's Tournament Matc...
3,2015,M,1352,1316,16,16,0,0.575758,69.000000,67.727273,1.272727,0.645161,73.935484,68.516129,5.419355,-0.069404,-4.146628,-4.935484,-0.788856,"Based on the provided data, it's somewhat surp..."
4,2015,M,1112,1411,2,15,13,0.911765,76.441176,58.617647,17.823529,0.647059,68.176471,67.441176,0.735294,0.264706,17.088235,8.264706,-8.823529,"1. **Does it make sense that team 1112 won, ba..."


# Narrative explanation

In [7]:
row = df.iloc[0]

prompt1 = f"""
Simulated NCAA Match – Season {row['Season']} ({row['Gender']})

Match Summary:
- Winner Team ID: {row['WTeamID']} — Seed {row['SeedA']}
- Loser Team ID: {row['LTeamID']} — Seed {row['SeedB']}
- Seed Difference: {row['SeedDiff']}

 Stats:
- Winner Win/Loss: {row['W_WinLoss']}, PPG: {row['W_PPG']}, Allowed: {row['W_PtsAllowed']}, Margin: {row['W_Margin']}
- Loser Win/Loss: {row['L_WinLoss']}, PPG: {row['L_PPG']}, Allowed: {row['L_PtsAllowed']}, Margin: {row['L_Margin']}

 Analysis task:
Write a short paragraph explaining why team {row['WTeamID']} likely won this match. Focus on the most important statistical factors.
"""
response1 = flash.generate_content(prompt1)
print(response1.text)

Despite having a slightly worse overall win/loss record and a lower points-per-game average (PPG) than team 1264, team 1214 emerged victorious due to a superior defense.  While team 1264 boasted a higher PPG and a positive scoring margin, team 1214 held their opponent to significantly fewer points than their own average allowed. This defensive strength, in holding team 1264 below their scoring average, proved to be the deciding factor in a closely contested matchup between two similarly-seeded teams.  The small margin of victory suggests a low-scoring, defensively-dominated game that team 1214 was better equipped to navigate.



#   Feature evaluation + suggestion

In [8]:
row = df.iloc[1]

prompt2 = f"""
You're analyzing a historical NCAA match simulation from Season {row['Season']} ({row['Gender']} division).

Teams:
- Winner: Team {row['WTeamID']} (Seed {row['SeedA']})
- Loser: Team {row['LTeamID']} (Seed {row['SeedB']})

Feature differences:
- Win/Loss ratio diff: {row['Diff_WinLoss']}
- Margin diff: {row['Diff_Margin']}
- PPG diff: {row['Diff_PPG']}
- Points allowed diff: {row['Diff_PtsAllowed']}

Tasks:
1. Does the winner make sense given this data?
2. Which feature seems most predictive in this case?
3. Suggest one additional stat that might improve the model's ability to predict match outcomes.
"""
response2 = flash.generate_content(prompt2)
print(response2.text)

1. **Does the winner make sense given this data?**

No, the winner doesn't entirely make sense given the data.  All four feature differences are negative, suggesting Team 1140 (the loser) was significantly stronger across those metrics.  A negative win/loss ratio difference indicates Team 1140 had a better win-loss record, a negative margin difference shows Team 1140 won by a larger average margin, a negative PPG difference shows Team 1140 scored more points per game, and a negative points allowed difference means Team 1140 conceded fewer points.  These are all strong indicators that Team 1140 should have won.  The fact that Team 1279 (seed 11) won despite being inferior across these metrics suggests there might be significant flaws in the model used for the simulation or that other unmeasured factors significantly influenced the outcome.


2. **Which feature seems most predictive in this case?**

While all features point in the same direction, the **PPG diff (-9.21875)** seems the mos